Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [15]:
train_subset = 10000
batch_size = 128
eta = 0.01
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32 , shape= (batch_size , image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32 , shape= (batch_size , num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.nn.relu(tf.matmul(tf_train_dataset , weights) + biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels , logits = logits) + eta * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediciton = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases))
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_test_dataset,weights) + biases))
    
num_steps = 40000

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Begin")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels =train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer , loss , train_prediciton] , feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Begin
Minibatch loss at step 0: 43.999657
Minibatch accuracy: 13.3%
Validation accuracy: 16.4%
Minibatch loss at step 500: 0.972500
Minibatch accuracy: 76.6%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.889017
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 0.665928
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 0.690426
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.568168
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 0.678183
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 3500: 0.672937
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 4000: 0.888898
Minibatch accuracy: 73.4%
Validation accuracy: 81.3%
Minibatch loss at step 4500: 0.633113
Minibatch accuracy: 85.2%
Validation accuracy: 81.2%
Minibatch los

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
train_subset = 10000
batch_size = 1000
eta = 0.01
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32 , shape= (batch_size , image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32 , shape= (batch_size , num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.nn.relu(tf.matmul(tf_train_dataset , weights) + biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels , logits = logits) + eta * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediciton = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases))
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_test_dataset,weights) + biases))
    
num_steps = 40000

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Begin")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels =train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer , loss , train_prediciton] , feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Begin
Minibatch loss at step 0: 46.134708
Minibatch accuracy: 7.4%
Validation accuracy: 9.7%
Minibatch loss at step 500: 0.832158
Minibatch accuracy: 84.5%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 0.703192
Minibatch accuracy: 82.6%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 0.698613
Minibatch accuracy: 82.2%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 0.696813
Minibatch accuracy: 83.1%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.663375
Minibatch accuracy: 83.8%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 0.696080
Minibatch accuracy: 82.7%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 0.633356
Minibatch accuracy: 84.5%
Validation accuracy: 82.4%
Minibatch loss at step 4000: 0.670595
Minibatch accuracy: 83.7%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 0.732545
Minibatch accuracy: 82.1%
Validation accuracy: 82.6%
Minibatch loss 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [20]:
train_subset = 10000
batch_size = 128
eta = 0.01
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32 , shape= (batch_size , image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32 , shape= (batch_size , num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.nn.relu(tf.nn.dropout(tf.matmul(tf_train_dataset , weights) + biases , 0.9))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels , logits = logits) + eta * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediciton = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases))
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_test_dataset,weights) + biases))
    
num_steps = 40000

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Begin")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels =train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer , loss , train_prediciton] , feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Begin
Minibatch loss at step 0: 45.878262
Minibatch accuracy: 9.4%
Validation accuracy: 8.8%
Minibatch loss at step 500: 1.136577
Minibatch accuracy: 72.7%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 1.105693
Minibatch accuracy: 71.1%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.939371
Minibatch accuracy: 73.4%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.914875
Minibatch accuracy: 72.7%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.774049
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.906120
Minibatch accuracy: 72.7%
Validation accuracy: 80.6%
Minibatch loss at step 3500: 0.979163
Minibatch accuracy: 70.3%
Validation accuracy: 81.2%
Minibatch loss at step 4000: 1.063095
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 4500: 0.808064
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [40]:
train_subset = 10000
batch_size = 128
eta = 0.005
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32 , shape= (batch_size , image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32 , shape= (batch_size , num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_l1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases_l1 = tf.Variable(tf.zeros([num_labels]))
    
    weights_l2 = tf.Variable(tf.truncated_normal([num_labels,num_labels]))
    biases_l2 = tf.Variable(tf.zeros([num_labels]))
    
    weights_l3 = tf.Variable(tf.truncated_normal([num_labels,num_labels]))
    biases_l3 = tf.Variable(tf.zeros([num_labels]))
    
    l1_R = tf.nn.relu(tf.matmul(tf_train_dataset , weights_l1) + biases_l1)
    l2_R = tf.nn.relu(tf.matmul(l1_R , weights_l2) + biases_l2)
    logits = tf.nn.relu(tf.matmul(l2_R , weights_l3) + biases_l3)
    
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels , logits = logits) + eta * tf.nn.l2_loss(weights_l1)+ eta * tf.nn.l2_loss(weights_l2)+ eta * tf.nn.l2_loss(weights_l3))
    
    optimizer = tf.train.GradientDescentOptimizer(0.08).minimize(loss)
    
    train_prediciton = tf.nn.softmax(logits)
    
    l1_VR = tf.nn.relu(tf.matmul(tf_valid_dataset , weights_l1) + biases_l1)
    l2_VR = tf.nn.relu(tf.matmul(l1_VR , weights_l2) + biases_l2)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(l2_VR , weights_l3) + biases_l3))
    
    l1_TR = tf.nn.relu(tf.matmul(tf_test_dataset , weights_l1) + biases_l1)
    l2_TR = tf.nn.relu(tf.matmul(l1_TR , weights_l2) + biases_l2)
    test_prediction  = tf.nn.softmax(tf.nn.relu(tf.matmul(l2_TR , weights_l3) + biases_l3))
    #valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases))
    #test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(tf_test_dataset,weights) + biases))
    
num_steps = 40000

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Begin")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels =train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer , loss , train_prediciton] , feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Begin
Minibatch loss at step 0: 79.687691
Minibatch accuracy: 11.7%
Validation accuracy: 10.7%
Minibatch loss at step 500: 11.836676
Minibatch accuracy: 42.2%
Validation accuracy: 49.8%
Minibatch loss at step 1000: 8.076452
Minibatch accuracy: 62.5%
Validation accuracy: 60.9%
Minibatch loss at step 1500: 5.631383
Minibatch accuracy: 62.5%
Validation accuracy: 65.7%
Minibatch loss at step 2000: 3.927679
Minibatch accuracy: 71.1%
Validation accuracy: 68.9%
Minibatch loss at step 2500: 2.859354
Minibatch accuracy: 74.2%
Validation accuracy: 70.6%
Minibatch loss at step 3000: 2.291668
Minibatch accuracy: 68.8%
Validation accuracy: 71.9%
Minibatch loss at step 3500: 1.710957
Minibatch accuracy: 77.3%
Validation accuracy: 72.8%
Minibatch loss at step 4000: 1.623525
Minibatch accuracy: 67.2%
Validation accuracy: 73.3%
Minibatch loss at step 4500: 1.267413
Minibatch accuracy: 74.2%
Validation accuracy: 73.7%
Minibatch lo

ValueError: Cannot use the default session to evaluate tensor: the tensor's graph is different from the session's graph. Pass an explicit session to `eval(session=sess)`.